# AgroSensor 3.0 Prototype - Database

In [2]:
%defaultDatasource jdbc:sqlite:./database/sample.sqlite

## Overview of the schema

![title](../resources/images/database-schema.png)

### Deleting existent data

In [7]:
DELETE FROM clients;
DELETE FROM farms;
DELETE FROM spots;

In [8]:
DELETE FROM devices;
DELETE FROM dimensions;

### Ensuring tables do not exists

In [3]:
DROP TABLE IF EXISTS clients;
DROP TABLE IF EXISTS farms;
DROP TABLE IF EXISTS spots;
DROP TABLE IF EXISTS devices;
DROP TABLE IF EXISTS dimensions;

## Creating tables and inserting initial data

### 1 - Clients

In [4]:
CREATE TABLE clients (
    id       integer    PRIMARY KEY AUTOINCREMENT,
    name     text       NOT NULL,
    email    text       NOT NULL  UNIQUE
);

In [5]:
INSERT INTO clients(name, email) VALUES
    ('Asdrubal Dino', "dino@acme.com"),
    ('Tereza Enzos',  "t.reza@ig.com")
;

In [6]:
SELECT * INTO ${clientsTable} FROM clients;

%%python
from beakerx import beakerx
print(beakerx.clientsTable)

   id           name          email
0   1  Asdrubal Dino  dino@acme.com
1   2   Tereza Enzos  t.reza@ig.com


### 2 - Farms

In [7]:
CREATE TABLE farms (
    id             integer       PRIMARY KEY AUTOINCREMENT,
    name           text          NOT NULL,
    description    text              NULL,
    owner_id       integer       NOT NULL,
    FOREIGN KEY    (owner_id)    REFERENCES clients(id)
);

In [8]:
INSERT INTO farms(name, owner_id) VALUES  
    ('Fazenda Antes Só do que Mal Acompanhado', 1),
    ('Rancho Melhor Ser Alegre do que Triste',  1),
    ('Fazenda Não Vai Subir Ninguém',           2),
    ('Fazenda Água Mole Pedra Dura',            1),
    ('Fazenda Pode? Poder, Pode...',            2),
    ('Fazenda o Dia Acaba com a Noite',         2)
;

In [9]:
SELECT * INTO ${farmsTable} FROM farms;

%%python
from beakerx import beakerx
print(beakerx.farmsTable)

   id                                     name description  owner_id
0   1  Fazenda Antes Só do que Mal Acompanhado        None         1
1   2   Rancho Melhor Ser Alegre do que Triste        None         1
2   3            Fazenda Não Vai Subir Ninguém        None         2
3   4             Fazenda Água Mole Pedra Dura        None         1
4   5             Fazenda Pode? Poder, Pode...        None         2
5   6          Fazenda o Dia Acaba com a Noite        None         2


### 3 - Spots

In [10]:
CREATE TABLE spots (
    id             integer      PRIMARY KEY AUTOINCREMENT,
    label          text         NOT NULL,
    farm_id        integer      NOT NULL,
    FOREIGN KEY    (farm_id)    REFERENCES farms(id)     
);

In [11]:
INSERT INTO spots(label, farm_id) VALUES  
    ('Aviário 1',          1),
    ('Aviário 1',          2),
    ('Aviário A',          3),
    ('Aviário lá de cima', 4),
    ('Aviário Blue House', 5),
    ('Aviário 1',          6),
    ('Crechário 1',        1),
    ('Crechário 2',        1),
    ('Crechário 3',        1)
;

In [12]:
SELECT * INTO ${spotsTable} FROM spots;

%%python
from beakerx import beakerx
print(beakerx.spotsTable)

   id               label  farm_id
0   1           Aviário 1        1
1   2           Aviário 1        2
2   3           Aviário A        3
3   4  Aviário lá de cima        4
4   5  Aviário Blue House        5
5   6           Aviário 1        6
6   7         Crechário 1        1
7   8         Crechário 2        1
8   9         Crechário 3        1


### 4 - Devices

In [13]:
CREATE TABLE devices (
    edgeid        text        NOT NULL,
    client_id     integer     NOT NULL,
    farm_id       integer     NOT NULL,
    thing_code    integer     NOT NULL,
    FOREIGN KEY(client_id)    REFERENCES clients(id),
    FOREIGN KEY(farm_id)      REFERENCES farms(id)   
    PRIMARY KEY(edgeid, client_id)
);

In [14]:
INSERT INTO devices(edgeid, client_id, farm_id, thing_code) VALUES
    ('2DPEQ572HEXP', 1, 1, 28),
    ('3WPH414SRSTZ', 1, 2, 28)
;

In [15]:
SELECT * INTO ${devicesTable} FROM devices;

%%python
from beakerx import beakerx
print(beakerx.devicesTable)

         edgeid  client_id  farm_id  thing_code
0  2DPEQ572HEXP          1        1          28
1  3WPH414SRSTZ          1        2          28


### 5 - Dimensions

In [16]:
CREATE TABLE dimensions (
    --id             integer      NOT NULL,
    edgeid           text         NOT NULL,
    port             integer      NOT NULL,
    sensor           integer      NOT NULL,
    dimension        integer      NOT NULL,
    spot_id          integer      NOT NULL,
    ts_from          integer      NOT NULL,
    ts_to            integer          NULL,
    last_value       float        NOT NULL,
    last_value_ts    float        NOT NULL,    
    FOREIGN KEY      (spot_id)    REFERENCES spots(id)    
    FOREIGN KEY      (edgeid)     REFERENCES devices(edgeid)    
    PRIMARY KEY      (edgeid, port, sensor, spot_id)
);

In [17]:
INSERT INTO 
dimensions(edgeid        ,port,sensor,dimension,spot_id,ts_from,ts_to,last_value,last_value_ts) VALUES  
          ('2DPEQ572HEXP',   4,     1,        1,      1,      0, null,     12345,1535745710000),
          ('2DPEQ572HEXP',   4,     2,        3,      1,      0, null,     12345,1535745710000),
          ('2DPEQ572HEXP',   5,     1,        1,      1,      0, null,     12345,1535745710000),
          ('2DPEQ572HEXP',   5,     2,        3,      1,      0, null,     12345,1535745710000)
;

In [18]:
SELECT * INTO ${dimensionsTable} FROM dimensions;

%%python
from beakerx import beakerx
print(beakerx.dimensionsTable)

         edgeid  port  sensor  dimension  spot_id  ts_from ts_to  last_value  \
0  2DPEQ572HEXP     4       1          1        1        0  None     12345.0   
1  2DPEQ572HEXP     4       2          3        1        0  None     12345.0   
2  2DPEQ572HEXP     5       1          1        1        0  None     12345.0   
3  2DPEQ572HEXP     5       2          3        1        0  None     12345.0   

   last_value_ts  
0   1.535746e+12  
1   1.535746e+12  
2   1.535746e+12  
3   1.535746e+12  
